# Übung 4

## Aufgabe 16
Erweitern Sie das logistische Regressionsmodell
$$M_1: Unfall∼Geschlecht+Beruf$$
für den Datensatz Unfaelle.txt um die Interaktion zwischen Geschlecht und
Beruf zum Modell
$$M_2: Unfall∼Geschlecht+Beruf+Geschlecht:Beruf$$
Die Interaktion von Geschlecht und Beruf sind zwei neue Variablen $GD_1$ und
$GD_2$, wobei $G$ die Dummy-Variable für Geschlecht und $D_1$, $D_1$ die Dummy-
Variablen für die Berufe Physiker und Zahnarzt sind. Benutzen Sie das Modell, um die Odds für einen Unfall der verschiedenen Berufe getrennt nach
Geschlecht zu schätzen. Welche inhaltliche Bedeutung hat die Interaktion zwischen Geschlecht und Beruf? Ist die Interaktion signifikant auf dem 5% Niveau?
Führen Sie für die Interaktion den Likelihood-Ratio-Test durch.

In [1]:
unfaelle <- read.csv(file = "../Uebung 3/Unfaelle.txt", sep=" ")
head(unfaelle)

,Lnr,Unfall,Geschlecht,Beruf,Alter,Fahrpraxis
,<int>,<int>,<chr>,<chr>,<int>,<int>
1,1,0,Mann,Physiker,31,12
2,2,0,Frau,Physiker,37,18
3,3,0,Frau,Physiker,38,18
4,4,0,Frau,Physiker,44,25
5,5,0,Frau,Physiker,52,33
6,6,0,Mann,Physiker,41,22


Zur Veranschaulichung das Modell $M_1$

In [2]:
Geschlecht_factor <- as.factor(unfaelle$Geschlecht)
Beruf_factor <- as.factor(unfaelle$Beruf)

model_base <- glm(Unfall ~ Geschlecht_factor + Beruf_factor, data=unfaelle, family=binomial)
coef(model_base)

(Intercept) Geschlecht_factorMann  Beruf_factorPhysiker 
           -3.0902793            -0.7936064            -0.4364080 
 Beruf_factorZahnarzt 
            0.7863344

Und $M_2$

In [3]:
model_extended <- glm(Unfall ~ Geschlecht_factor + Beruf_factor + Geschlecht_factor : Beruf_factor, data=unfaelle, family=binomial)
coef(model_extended)

(Intercept) 
                                -3.1581196 
                     Geschlecht_factorMann 
                                -0.5199877 
                      Beruf_factorPhysiker 
                                -0.3330260 
                      Beruf_factorZahnarzt 
                                 0.8944644 
Geschlecht_factorMann:Beruf_factorPhysiker 
                                -0.4196834 
Geschlecht_factorMann:Beruf_factorZahnarzt 
                                -0.4352328

**Odds:**

Allgemein gilt:
$$
\log \frac{1}{1-p} = \beta_0 + \beta_1 \text{Mann} + ... \Rightarrow \frac{1}{1-p} = exp(\beta_0 + \beta_1 \text{Mann} + ...)
$$

Also müssen wir die Exponentialfunktion auf die jeweiligen Koeffizienten anwenden

In [4]:
coefficients <- coef(model_extended)

beta0 <- coefficients[1]
beta1 <- coefficients[2] # GeschlechtMann
beta2 <- coefficients[3] # BerufPhysiker
beta3 <- coefficients[4] # BerufZahnarzt
beta4 <- coefficients[5] # GeschlechtMann:BerufPhysiker (zusätzliches Risiko, falls ein Mann Physiker ist, im Vergleich zu einer Frau)
beta5 <- coefficients[6] # GeschlechtMann:BerufZahnarzt (analog)

logit_mann_bio <- beta0 + beta1
logit_mann_phys <- beta0 + beta1 + beta4
logit_mann_zahn <- beta0 + beta1 + beta5
logit_frau_bio <- beta0
logit_frau_phys <- beta0 + beta2
logit_frau_zahn <- beta0 + beta3

odds_mann_bio <- exp(logit_mann_bio)
odds_mann_phys <- exp(logit_mann_phys)
odds_mann_zahn <- exp(logit_mann_zahn)
odds_frau_bio <- exp(logit_frau_bio)
odds_frau_phys <- exp(logit_frau_phys)
odds_frau_zahn <- exp(logit_frau_zahn)

cat("Biologe: ", odds_mann_bio, "\nPhysiker: ", odds_mann_phys, "\nZahnarzt: ", odds_mann_zahn, "\nBiologin: ", odds_frau_bio, "\nPhysikerin", odds_frau_phys, "\nZahnärztin: ", odds_frau_zahn)

Biologe:  0.02527076 
Physiker:  0.01660933 
Zahnarzt:  0.01635306 
Biologin:  0.04250559 
Physikerin 0.03046595 
Zahnärztin:  0.1039698

Die Interaktion zwischen Geschlecht und Beruf:

In [5]:
summary(model_extended)


Call:
glm(formula = Unfall ~ Geschlecht_factor + Beruf_factor + Geschlecht_factor:Beruf_factor, 
    family = binomial, data = unfaelle)

Coefficients:
                                           Estimate Std. Error z value Pr(>|z|)
(Intercept)                                 -3.1581     0.1656 -19.067  < 2e-16
Geschlecht_factorMann                       -0.5200     0.3173  -1.639    0.101
Beruf_factorPhysiker                        -0.3330     0.2967  -1.122    0.262
Beruf_factorZahnarzt                         0.8945     0.2180   4.104 4.06e-05
Geschlecht_factorMann:Beruf_factorPhysiker  -0.4197     0.6030  -0.696    0.486
Geschlecht_factorMann:Beruf_factorZahnarzt  -0.4352     0.4310  -1.010    0.313
                                              
(Intercept)                                ***
Geschlecht_factorMann                         
Beruf_factorPhysiker                          
Beruf_factorZahnarzt                       ***
Geschlecht_factorMann:Beruf_factorPhysiker    
Gesch

In diesem Fall bedeuten:
- **Geschlecht_factorMann:Beruf_factorPhysiker ($\beta_4$)** Die Veränderung der Log-Odds eines Unfalls für Männer, die Physiker sind, im Vergleich zu Frauen, die Physiker sind, zusätzlich zu allen anderen Faktoren
- **Geschlecht_factorMann:Beruf_factorZahnarzt ($\beta_5$)** Die Veränderung der Log-Odds eines Unfalls für Männer, die Zahnärzte sind, im Vergleich zu Frauen, die Zahnärzte sind, zusätzlich zu allen anderen Faktoren

In der Tabelle sind zusätzlich unter `Pr(>|z|)` die jeweiligen p-Werte zu finden.

Bei $\beta_4$ findet sich ein p-Wert von $0.486$. Das bedeutet, dass wir die Nullhypothese nicht ablehnen. Anders gesagt: Der Unterschied der Log-Odds eines Unfalls zwischen Physikern und Physikerinnen ist **nicht** groß genug, um statistisch signifikant zu sein.

Bei $beta_5$ ist der p-Wert ebenfalls weit über die $5\%$-Grenze, weswegen wir bei Zahnärzten ebenfalls keinen signifikanten Unterschied beobachten können.

Als letztes führen wir den Likelihood-Ratio-Test durch:

In [6]:
lrtest_result <- anova(model_base, model_extended, test = "LRT")
print(lrtest_result)

Analysis of Deviance Table

Model 1: Unfall ~ Geschlecht_factor + Beruf_factor
Model 2: Unfall ~ Geschlecht_factor + Beruf_factor + Geschlecht_factor:Beruf_factor
  Resid. Df Resid. Dev Df Deviance Pr(>Chi)
1      3496     1157.8                     
2      3494     1156.7  2   1.1118   0.5736


Der p-Wert ist **nicht** signifikant ($<5\%$), also ist unser erweitertes Model $M_2$ nicht besser als unser Ausgangsmodell $M_1$, um die Log-Odds eines Unfalls zu schätzen.